# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint
# Import API key
from api_keys import g_key

%matplotlib inline

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
# Read the data
weather_df = pd.read_csv("../WeatherPy/weather.csv",index_col=0)

weather_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,ushuaia,-54.8000,-68.3000,50.00,66,75,4.61,AR,1616845069
1,kapaa,22.0752,-159.3190,73.40,83,1,13.80,US,1616845159
2,dikson,73.5069,80.5464,-11.07,98,28,8.70,RU,1616845160
3,cape town,-33.9258,18.4232,72.00,52,20,18.41,ZA,1616844916
4,atuona,-9.8000,-139.0333,80.02,79,86,19.37,PF,1616845161


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
#Gmaps
gmaps.configure(api_key = g_key)
# Heatmap location
location = weather_df[["Lat","Lng"]]
humidity = weather_df["Humidity"].astype(float)
print(humidity.max())

100.0


In [11]:
#Humidity heatmap
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(location,
                                 weights=humidity,
                                dissipating = False,
                                max_intensity = 100,
                                point_radius = 2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
# * Narrow down the DataFrame to find ideal weather condition for my vacation.
#   * A max temperature lower than 100 degrees but higher than 80.
#   * Wind speed up to 15 mph.
#   * Humidity from 40% to 65%
weather_df.loc[weather_df["Max Temp"]>80]
weather_df["my_temp"] = weather_df["Max Temp"].between(80, 100, inclusive = False)
weather_df["my_wind"] = weather_df["Wind Speed"].between(0, 15, inclusive = True)
weather_df["my_humidity"] = weather_df["Humidity"].between(40, 65, inclusive = True)

weather_df
my_places =[]
for row in range(len(weather_df)):
    if weather_df.iloc[row,9] == True:
        if weather_df.iloc[row,10] == True:
            if weather_df.iloc[row,11] == True:
                my_places.append(weather_df.iloc[row,:])
                
my_place_df = pd.DataFrame(my_places)

#Drop any rows will null values.
my_place_df.dropna()    
my_place_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,my_temp,my_wind,my_humidity
57,georgetown,5.4112,100.3354,89.60,62,20,8.05,MY,1616845192,True,True,True
77,koungou,-12.7336,45.2042,87.80,62,20,9.22,YT,1616845204,True,True,True
107,matadi,-5.8167,13.4500,88.72,61,100,5.79,CD,1616845222,True,True,True
110,inhambane,-23.8650,35.3833,86.00,62,75,6.91,MZ,1616845224,True,True,True
112,bandarbeyla,9.4942,50.8122,83.14,62,99,13.15,SO,1616845225,True,True,True
146,ranong,9.9658,98.6348,89.60,58,20,12.66,TH,1616845245,True,True,True
149,beira,-19.8436,34.8389,91.40,58,20,10.36,MZ,1616845246,True,True,True
174,kampene,-3.6000,26.6667,87.84,43,94,4.25,CD,1616845262,True,True,True
175,mattru,7.6244,-11.8332,95.65,43,71,2.48,SL,1616845262,True,True,True
211,santa marta,11.2408,-74.1990,81.00,65,20,4.61,CO,1616845282,True,True,True


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
hotel_df = my_place_df[["City","Lat","Lng","Country"]]

In [9]:
hotel_df["Hotel Name"] = "" 
hotel_df

<ipython-input-9-035905bd69f7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Lat,Lng,Country,Hotel Name
14,kaduna,10.5222,7.4383,NG,
47,santiago del estero,-27.7951,-64.2615,AR,
82,kavaratti,10.5669,72.6420,IN,
84,guadalupe,25.6833,-100.2500,MX,
105,hithadhoo,-0.6000,73.0833,MV,
157,kieta,-6.2167,155.6333,PG,
197,voh,-20.9667,164.7000,NC,
203,paita,-5.0892,-81.1144,PE,
245,poya,-21.3500,165.1500,NC,
305,faanui,-16.4833,-151.7500,PF,


In [19]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    name_address = requests.get(base_url, params=params)
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "not found"
hotel_df

C:\Users\meria\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,City,Lat,Lng,Country,Hotel Name
57,georgetown,5.4112,100.3354,MY,Cititel Penang
77,koungou,-12.7336,45.2042,YT,Villa Netibor
107,matadi,-5.8167,13.4500,CD,Hotel Fortune
110,inhambane,-23.8650,35.3833,MZ,Oceano
112,bandarbeyla,9.4942,50.8122,SO,JABIR HOTEL
146,ranong,9.9658,98.6348,TH,Tinidee Hotel@Ranong
149,beira,-19.8436,34.8389,MZ,Beira Terrace Hotel
174,kampene,-3.6000,26.6667,CD,not found
175,mattru,7.6244,-11.8332,SL,"Mondkor Town, Bagbo Chiefdom, Bo District"
211,santa marta,11.2408,-74.1990,CO,Casa de Isabella


In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [21]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations,info_box_content = hotel_info)

# Display figure
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))